In [ ]:
import wave, numpy as np
from IPython.display import Audio, display

In [ ]:
with wave.open("Noisy.wav", "rb") as w:
    fr = w.getframerate()
    ch = w.getnchannels()
    sw = w.getsampwidth()
    n  = w.getnframes()
    raw = w.readframes(n)

# Read 16-bit stereo audio bytes into numbers, then average left and right to get a mono waveform.
data = np.frombuffer(raw, dtype=np.int16)
if ch == 2:  # downmix to mono, also means that there is audio from left and right so we split those data[:,0] = left  data[:,1] = right
    data = data.reshape(-1, 2).mean(axis=1).astype(np.int16) # avg of left and right for each step, one combined signal instead of two, goes back to 16-bit integer

# float in [-1,1] for DSP
x = data.astype(np.float32) / 32768.0
print(f"fs={fr} Hz, samples={x.size}")


fs=16000 Hz, samples=102059


In [ ]:
# displays the original audio for faater hearing
display(Audio(x, rate=fr))

In [ ]:
# detect noisy tone here, FFT: detect dominant noisy tone
N = x.size
X = np.fft.rfft(x)
freqs = np.fft.rfftfreq(N, d=1.0/fr)

mag = np.abs(X)
peak_idx = np.argmax(mag[1:]) + 1   # skip DC
peak_freq = freqs[peak_idx]
print(f"Dominant tone ≈ {peak_freq:.2f} Hz (bin {peak_idx})")


Dominant tone ≈ 4000.04 Hz (bin 25515)


In [72]:
# “Remove the noisy tone by replacing its amplitude by zero and do the inverse FFT”
width_hz = 85.0      # change to increase or decrease amount of noise.
band = np.where(np.abs(freqs - peak_freq) <= width_hz)[0] # finds all frequency indices where the frequency is within ± width_hz of that tone.
# freq = actual frequencies in hz to each fftbin 
# peak_freq = loudest unwanted tone

Xc = X.copy() # clone so dont destroy the orignal
Xc[band] = 0 # remove the noise
xc = np.fft.irfft(Xc, n=N).astype(np.float32) # inverse fft 

In [74]:
# save clean wav here

y = np.clip(xc, -1.0, 1.0)
y_i16 = (y * 32767.0).astype(np.int16)

with wave.open("clean.wav", "wb") as w:
    w.setnchannels(1)
    w.setsampwidth(2)   # 16-bit
    w.setframerate(fr)
    w.writeframes(y_i16.tobytes())

print(f"Saved: clean.wav")


Saved: clean.wav


In [75]:
display(Audio(y, rate=fr))

# width_hz
- helps remove the noise when u increase it but also risks removing the real noise
    - but i increase to 2000 and the noise is gone but the voice sounds a bit echoey
# its also much better to play it in ipynb and not the clean.wav file for louder hearing